In [1]:
!pip install datasets


In [2]:
import os
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"

from datasets import load_dataset


# Load datasets
dataset1 = load_dataset("dux-tecblic/symptom-disease-dataset")
dataset2 = load_dataset("prognosis/symptoms_disease_v1")
dataset3 = load_dataset("QuyenAnhDE/Diseases_Symptoms")
dataset4 = load_dataset("gretelai/symptom_to_diagnosis")
dataset5 = load_dataset("fhai50032/Symptoms_to_disease_7k")
dataset6 = load_dataset("Dhananjay-25/symptoms-disease_dataset_for_LLM")
dataset7 = load_dataset("celikmus/mayo_clinic_symptoms_and_diseases_v1")

# Display one record from each
print("\n[dataset1]", dataset1['train'][0])
print("\n[dataset2]", dataset2['train'][0])
print("\n[dataset3]", dataset3['train'][0])
print("\n[dataset4]", dataset4['train'][0])
print("\n[dataset5]", dataset5['train'][0])
print("\n[dataset6]", dataset6['train'][0])
print("\n[dataset7]", dataset7['train'][0])




Repo card metadata block was not found. Setting CardData to empty.



[dataset1] {'text': "I have been having migraines and headaches. I can't sleep. My whole body is shaking and shivering. I feel dizzy sometimes.", 'label': 308}

[dataset2] {'input': '', 'reference': '', 'output': 'The following are the symptoms of hypertensive  disease: pain  chest, shortness  of breath, dizziness, asthenia, fall, syncope, vertigo, sweating  increased, palpitation, nausea, angina  pectoris, pressure  chest', 'instruction': 'What are the symptoms of hypertensive  disease?'}

[dataset3] {'Code': 1, 'Name': 'Panic disorder', 'Symptoms': 'Palpitations, Sweating, Trembling, Shortness of breath, Fear of losing control, Dizziness', 'Treatments': 'Antidepressant medications, Cognitive Behavioral Therapy, Relaxation Techniques'}

[dataset4] {'output_text': 'cervical spondylosis', 'input_text': "I've been having a lot of pain in my neck and back. I've also been having trouble with my balance and coordination. I've been coughing a lot and my limbs feel weak."}

[dataset5] {'quer

In [3]:
# Cleaning functions for all 7 datasets

def clean_dataset1(example):
    return {
        "symptoms": example["text"],
        "diagnosis": str(example["label"])
    }

def clean_dataset2(example):
    return {
        "symptoms": example["instruction"],
        "diagnosis": example["output"]
    }

def clean_dataset3(example):
    return {
        "symptoms": example["Symptoms"],
        "diagnosis": example["Name"]
    }

def clean_dataset4(example):
    return {
        "symptoms": example["input_text"],
        "diagnosis": example["output_text"]
    }

def clean_dataset5(example):
    return {
        "symptoms": example["query"],
        "diagnosis": example["response"]
    }

def clean_dataset6(example):
    return {
        "symptoms": example["instruction"],
        "diagnosis": example["output"]
    }

def clean_dataset7(example):
    return {
        "symptoms": example["text"],
        "diagnosis": example["label"]
    }
    
cleaned1 = dataset1["train"].map(clean_dataset1, remove_columns=dataset1["train"].column_names)
cleaned2 = dataset2["train"].map(clean_dataset2, remove_columns=dataset2["train"].column_names)
cleaned3 = dataset3["train"].map(clean_dataset3, remove_columns=dataset3["train"].column_names)
cleaned4 = dataset4["train"].map(clean_dataset4, remove_columns=dataset4["train"].column_names)
cleaned5 = dataset5["train"].map(clean_dataset5, remove_columns=dataset5["train"].column_names)
cleaned6 = dataset6["train"].map(clean_dataset6, remove_columns=dataset6["train"].column_names)
cleaned7 = dataset7["train"].map(clean_dataset7, remove_columns=dataset7["train"].column_names)



In [4]:
print(cleaned1[0]) 
print(cleaned2[0])


{'symptoms': "I have been having migraines and headaches. I can't sleep. My whole body is shaking and shivering. I feel dizzy sometimes.", 'diagnosis': '308'}
{'symptoms': 'What are the symptoms of hypertensive  disease?', 'diagnosis': 'The following are the symptoms of hypertensive  disease: pain  chest, shortness  of breath, dizziness, asthenia, fall, syncope, vertigo, sweating  increased, palpitation, nausea, angina  pectoris, pressure  chest'}


In [5]:
from datasets import concatenate_datasets

all_data = concatenate_datasets([
    cleaned1, cleaned2, cleaned3, cleaned4, cleaned5, cleaned6, cleaned7
])


In [6]:
# Only once after loading
all_data_list = all_data.to_list()


In [80]:
# Your diagnosis explanation mapping
diagnosis_map = {
     35: "These symptoms may indicate a bacterial infection or flu-like illness. Rest and medical attention are recommended.",
    149: "Your symptoms could be related to a viral upper respiratory infection. Monitor your condition and seek medical advice if it worsens.",
    768: "You may be experiencing symptoms of angina or a heart-related issue. Please consult a cardiologist.",
    101: "These symptoms are commonly related to fever. Stay hydrated and monitor your temperature.",
    212: "This might be a sign of seasonal allergies or a respiratory condition.",
    305: "You may be showing signs of a gastrointestinal issue. Please consult a physician.",
    872: "These symptoms may suggest a cardiovascular condition. Please seek immediate medical attention.",
    404: "These symptoms are associated with a viral infection. Consider rest and over-the-counter medication.",
    509: "Possible early signs of diabetes. It's recommended to get your blood sugar levels checked.",
    612: "May be due to stress or anxiety. Practicing relaxation techniques or talking to a counselor may help.",
    123: "Could indicate dehydration or low blood pressure. Increase fluid intake and monitor symptoms.",
    321: "Symptoms suggest a respiratory tract infection. A visit to a general practitioner is advised.",
    111: "This may be a mild cold or flu. Rest and hydration are typically helpful.",
    777: "Unusual symptoms. Please seek medical attention for a detailed diagnosis.",
    513: "You may be showing signs of asthma or another respiratory condition. Please consult a doctor.",
    308: "These symptoms may point to a sinus infection or allergic rhinitis. Try antihistamines and consult a doctor if it persists.",
    35: "These symptoms may indicate a bacterial infection or flu-like illness. Rest and medical attention are recommended.",
    149: "Your symptoms could be related to a viral upper respiratory infection. Monitor your condition and seek medical advice if it worsens.",
    200: "Symptoms suggest food poisoning. Stay hydrated and avoid solid food until symptoms improve.",
    266: "Signs of urinary tract infection detected. Increased fluid intake and medical evaluation are suggested.",
    314: "You may be experiencing a migraine. Rest in a quiet, dark room and consider over-the-counter medication.",
    401: "Possible case of anemia. A blood test and iron supplements might be necessary.",
    688: "This could be due to hormonal imbalance. Consult an endocrinologist.",
    809: "Possible allergic reaction. Avoid allergens and seek antihistamines or medical advice if symptoms worsen.",
    902: "These symptoms match bronchitis. Rest and a medical checkup are advisable.",
    999: "Severe symptoms detected. Seek emergency medical attention immediately."
}


def find_diagnosis_flexible(user_input, dataset, diagnosis_map):
    user_words = user_input.lower().split()

    for entry in dataset:
        symptoms = entry["symptoms"].lower()
        for word in user_words:
            if word in symptoms:
                diag_id = int(entry["diagnosis"])
                return diagnosis_map.get(diag_id, f"Diagnosis ID: {diag_id} (No explanation available)")
    
    return "Sorry, no similar symptoms found."

# ✅ Example usage
#user_input = "runny nose and sneezing"
user_input = "persistent cough"
diagnosis = find_diagnosis_flexible(user_input, all_data, diagnosis_map)
print(diagnosis)


Your symptoms could be related to a viral upper respiratory infection. Monitor your condition and seek medical advice if it worsens.
